In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.types import  Row
from datetime import datetime

In [3]:
sc =SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()

In [4]:
spark = SparkContext.getOrCreate()

In [5]:
stu = spark.parallelize([Row(roll_no=1,name='John Doe',passed=True,marks={'Math':89,'Physics':87,'Chemistry':81},sports =['chess','football'], DoB=datetime(2012,5,1,13,1,5)), Row(roll_no=2,name='John Smith',passed=False,marks={'Math':29,'Physics':31,'Chemistry':36}, sports =['volleyball','tabletennis'], DoB=datetime(2012,5,12,14,2,5))])

In [6]:
student_rec = stu.toDF()
student_rec.show()

+-------+----------+------+--------------------+--------------------+-------------------+
|roll_no|      name|passed|               marks|              sports|                DoB|
+-------+----------+------+--------------------+--------------------+-------------------+
|      1|  John Doe|  true|{Chemistry -> 81,...|   [chess, football]|2012-05-01 13:01:05|
|      2|John Smith| false|{Chemistry -> 36,...|[volleyball, tabl...|2012-05-12 14:02:05|
+-------+----------+------+--------------------+--------------------+-------------------+



truncate will show full content of the table

In [7]:
student_rec.show(truncate=False)

+-------+----------+------+--------------------------------------------+-------------------------+-------------------+
|roll_no|name      |passed|marks                                       |sports                   |DoB                |
+-------+----------+------+--------------------------------------------+-------------------------+-------------------+
|1      |John Doe  |true  |{Chemistry -> 81, Math -> 89, Physics -> 87}|[chess, football]        |2012-05-01 13:01:05|
|2      |John Smith|false |{Chemistry -> 36, Math -> 29, Physics -> 31}|[volleyball, tabletennis]|2012-05-12 14:02:05|
+-------+----------+------+--------------------------------------------+-------------------------+-------------------+



In [8]:
student_rec.createOrReplaceTempView('records')

In [9]:
sc.sql("SELECT * FROM records").show()

+-------+----------+------+--------------------+--------------------+-------------------+
|roll_no|      name|passed|               marks|              sports|                DoB|
+-------+----------+------+--------------------+--------------------+-------------------+
|      1|  John Doe|  true|{Chemistry -> 81,...|   [chess, football]|2012-05-01 13:01:05|
|      2|John Smith| false|{Chemistry -> 36,...|[volleyball, tabl...|2012-05-12 14:02:05|
+-------+----------+------+--------------------+--------------------+-------------------+



##### Accessing elements and records inside a column

In [10]:
sc.sql('SELECT roll_no, marks["Physics"] ,sports[1] FROM records').show()

+-------+--------------+-----------+
|roll_no|marks[Physics]|  sports[1]|
+-------+--------------+-----------+
|      1|            87|   football|
|      2|            31|tabletennis|
+-------+--------------+-----------+



###### Where Clause

In [16]:
sc.sql('SELECT * FROM records WHERE passed = True').show()

+-------+--------+------+--------------------+-----------------+-------------------+
|roll_no|    name|passed|               marks|           sports|                DoB|
+-------+--------+------+--------------------+-----------------+-------------------+
|      1|John Doe|  true|{Chemistry -> 81,...|[chess, football]|2012-05-01 13:01:05|
+-------+--------+------+--------------------+-----------------+-------------------+



In [17]:
sc.sql('SELECT * FROM records WHERE marks["Chemistry"] < 40').show()

+-------+----------+------+--------------------+--------------------+-------------------+
|roll_no|      name|passed|               marks|              sports|                DoB|
+-------+----------+------+--------------------+--------------------+-------------------+
|      2|John Smith| false|{Chemistry -> 36,...|[volleyball, tabl...|2012-05-12 14:02:05|
+-------+----------+------+--------------------+--------------------+-------------------+



the view that we have created above have only scope on current session. in order to make it as global view session we use  the following command

In [18]:
student_rec.createGlobalTempView("Global_rec")
sc.sql("SELECT * FROM global_temp.Global_rec").show()

+-------+----------+------+--------------------+--------------------+-------------------+
|roll_no|      name|passed|               marks|              sports|                DoB|
+-------+----------+------+--------------------+--------------------+-------------------+
|      1|  John Doe|  true|{Chemistry -> 81,...|   [chess, football]|2012-05-01 13:01:05|
|      2|John Smith| false|{Chemistry -> 36,...|[volleyball, tabl...|2012-05-12 14:02:05|
+-------+----------+------+--------------------+--------------------+-------------------+



###### To see the columns

In [19]:
student_rec.columns

['roll_no', 'name', 'passed', 'marks', 'sports', 'DoB']

###### To Drop the columns we dont need

In [20]:
student_rec.drop('passed')

DataFrame[roll_no: bigint, name: string, marks: map<string,bigint>, sports: array<string>, DoB: timestamp]

###### Average of the marks

In [24]:
sc.sql("SELECT *, round( (marks.Physics+marks.Chemistry+marks.Math)/3) avg_marks FROM records").show()

+-------+----------+------+--------------------+--------------------+-------------------+---------+
|roll_no|      name|passed|               marks|              sports|                DoB|avg_marks|
+-------+----------+------+--------------------+--------------------+-------------------+---------+
|      1|  John Doe|  true|{Chemistry -> 81,...|   [chess, football]|2012-05-01 13:01:05|     86.0|
|      2|John Smith| false|{Chemistry -> 36,...|[volleyball, tabl...|2012-05-12 14:02:05|     32.0|
+-------+----------+------+--------------------+--------------------+-------------------+---------+



##### Adding the avg column to the existing table 

In [25]:
student_rec = sc.sql('SELECT *, round( (marks.Physics+marks.Chemistry+marks.Math)/3) avg_marks FROM records')
student_rec.show()

+-------+----------+------+--------------------+--------------------+-------------------+---------+
|roll_no|      name|passed|               marks|              sports|                DoB|avg_marks|
+-------+----------+------+--------------------+--------------------+-------------------+---------+
|      1|  John Doe|  true|{Chemistry -> 81,...|   [chess, football]|2012-05-01 13:01:05|     86.0|
|      2|John Smith| false|{Chemistry -> 36,...|[volleyball, tabl...|2012-05-12 14:02:05|     32.0|
+-------+----------+------+--------------------+--------------------+-------------------+---------+



 inserting new column in order to find the status of the student pass or fail  

First we must update new view again

In [26]:
student_rec.createOrReplaceTempView("records")

In [27]:
student_rec = student_rec.withColumn('status', (when(col("avg_marks")>=40,"passed")).otherwise("failed"))
student_rec.show()

+-------+----------+------+--------------------+--------------------+-------------------+---------+------+
|roll_no|      name|passed|               marks|              sports|                DoB|avg_marks|status|
+-------+----------+------+--------------------+--------------------+-------------------+---------+------+
|      1|  John Doe|  true|{Chemistry -> 81,...|   [chess, football]|2012-05-01 13:01:05|     86.0|passed|
|      2|John Smith| false|{Chemistry -> 36,...|[volleyball, tabl...|2012-05-12 14:02:05|     32.0|failed|
+-------+----------+------+--------------------+--------------------+-------------------+---------+------+



#### Reading A CSV file

In [11]:
df =sc.read.format("csv").option("header","true").option("mode", "DROPMALFORMED").load("titanic_train.csv")


In [12]:
df.show(truncate=True)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [13]:
df.createOrReplaceTempView("newrec")

In [14]:
sc.sql("SELECT * FROM newrec").show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      